![alt text](../utils/conve.png)

In [1]:
from tqdm import tqdm

import tensorflow as tf
import numpy as np
import sklearn
import pprint
import itertools

import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

from data import WN18

In [2]:
class Config:
    n_epochs = 50
    batch_size = 120
    embed_dim = 150

In [3]:
"""
e: entity
s: subject
p: predicate
o: object
"""

def read_triples(path):
    triples = []
    with open(path, 'rt') as f:
        for line in f.readlines():
            s, p, o = line.split()
            triples += [(s.strip(), p.strip(), o.strip())]
    return triples


def load_triple():
    WN18.download()
    triples_tr = read_triples('../data/WN18/wn18/train.txt')
    triples_va = read_triples('../data/WN18/wn18/valid.txt')
    triples_te = read_triples('../data/WN18/wn18/test.txt')
    
    triples_all = triples_tr + triples_va + triples_te
    
    return triples_all, triples_tr, triples_va, triples_te


def build_vocab(triples):
    params = {}
    
    e_set = {s for (s, p, o) in triples} | {o for (s, p, o) in triples}
    p_set = {p for (s, p, o) in triples}

    params['e_vocab_size'] = len(e_set)
    params['p_vocab_size'] = len(p_set)

    e2idx = {e: idx for idx, e in enumerate(sorted(e_set))}
    p2idx = {p: idx for idx, p in enumerate(sorted(p_set))}
    
    return e2idx, p2idx, params


def build_multi_label(triples_tr):
    s2p2o = {}
    for (_s, _p, _o) in triples_tr:
        s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
        if s not in s2p2o:
            s2p2o[s] = {p: [o]}
        if p not in s2p2o[s]:
            s2p2o[s][p] = [o]
        if o not in s2p2o[s][p]:
            s2p2o[s][p].append(o)
    return s2p2o


def next_train_batch(triples_tr, e2idx, p2idx, s2p2o):
    for i in range(0, len(triples_tr), Config.batch_size):
        _triples_tr = triples_tr[i : i+Config.batch_size]
        x_s = np.asarray([e2idx[s] for (s, p, o) in _triples_tr], dtype=np.int32)
        x_p = np.asarray([p2idx[p] for (s, p, o) in _triples_tr], dtype=np.int32)
        y = []
        for (_s, _p, _o) in _triples_tr:
            s, p = e2idx[_s], p2idx[_p]
            temp = np.zeros([len(e2idx)])
            temp[np.random.choice(s2p2o[s][p], size=len(s2p2o[s][p]))] = 1.
            y.append(temp)
        y = np.asarray(y)
        yield x_s, x_p, y


def train_input_fn(triples_tr, e2idx, p2idx, s2p2o):
    dataset = tf.data.Dataset.from_generator(
        lambda: next_train_batch(sklearn.utils.shuffle(triples_tr),
                                 e2idx,
                                 p2idx,
                                 s2p2o),
        (tf.int32, tf.int32, tf.float32),
        (tf.TensorShape([None]),
         tf.TensorShape([None]),
         tf.TensorShape([None, len(e2idx)])))
    iterator = dataset.make_one_shot_iterator()
    x_s, x_p, y = iterator.get_next()
    return {'s': x_s, 'p': x_p}, y

In [4]:
def o_next_batch(eval_triples,
                 entity_to_idx, 
                 predicate_to_idx):
    for _i, (s, p, o) in tqdm(enumerate(eval_triples), total=len(eval_triples), ncols=70):
        s_idx, p_idx = entity_to_idx[s], predicate_to_idx[p]
        yield np.atleast_1d(s_idx), np.atleast_1d(p_idx)


def o_input_fn(eval_triples,
               entity_to_idx, 
               predicate_to_idx):
    dataset = tf.data.Dataset.from_generator(
        lambda: o_next_batch(eval_triples,
                             entity_to_idx, 
                             predicate_to_idx),
        (tf.int32, tf.int32),
        (tf.TensorShape([None,]),
         tf.TensorShape([None,])))
    iterator = dataset.make_one_shot_iterator()
    s, p = iterator.get_next()
    return {'s': s, 'p': p}


def evaluate_rank(model,
                  valid_triples,
                  test_triples,
                  all_triples,
                  entity_to_idx,
                  predicate_to_idx):

    for eval_name, eval_triples in [('test', test_triples)]:
        _scores_o = list(model.predict(
            lambda: o_input_fn(eval_triples,
                               entity_to_idx, 
                               predicate_to_idx)))

        ScoresO = np.reshape(_scores_o, [len(eval_triples), len(entity_to_idx)])
        ranks_o, filtered_ranks_o = [], []
    
        for _i, ((s, p, o), scores_o) in tqdm(enumerate(zip(eval_triples, ScoresO)),
                                              total=len(eval_triples),
                                              ncols=70):
            s_idx, p_idx, o_idx = entity_to_idx[s], predicate_to_idx[p], entity_to_idx[o]

            ranks_o += [1 + np.sum(scores_o > scores_o[o_idx])]
            
            filtered_scores_o = scores_o.copy()
            rm_idx_o = [entity_to_idx[fo] for (fs, fp, fo) in all_triples if fs == s and fp == p and fo != o]
            filtered_scores_o[rm_idx_o] = - np.inf
            
            filtered_ranks_o += [1 + np.sum(filtered_scores_o > filtered_scores_o[o_idx])]

        for setting_name, setting_ranks in [('Raw', ranks_o), ('Filtered', filtered_ranks_o)]:
            mean_rank = np.mean(setting_ranks)
            print('[{}] {} Mean Rank: {}'.format(eval_name, setting_name, mean_rank))
            for k in [1, 3, 5, 10]:
                hits_at_k = np.mean(np.asarray(setting_ranks) <= k) * 100
                print('[{}] {} Hits@{}: {}'.format(eval_name, setting_name, k, hits_at_k))

In [ ]:
def forward(features, mode, params):
    batch_sz = tf.shape(features['s'])[0]
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    
    e_embed = tf.get_variable('e_embed',
                              [params['e_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer())
    p_embed = tf.get_variable('p_embed',
                              [params['p_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer())
    
    s = tf.nn.embedding_lookup(e_embed, features['s'])
    p = tf.nn.embedding_lookup(p_embed, features['p'])
    
    x = tf.concat([tf.reshape(s, [batch_sz, 10, 15, 1]),
                   tf.reshape(p, [batch_sz, 10, 15, 1]),], axis=1)
    
    x = tf.layers.dropout(x, 0.2, training=is_training)
    x = tf.layers.conv2d(x, 32, (3,3), activation=tf.nn.relu)
    
    x = tf.layers.flatten(x)
    x = tf.layers.dropout(x, 0.2, training=is_training)
    x = tf.layers.dense(x, Config.embed_dim, tf.nn.relu)
    
    logits = tf.matmul(x, e_embed, transpose_b=True)
    return logits
    
    
def model_fn(features, labels, mode, params):
    logits = forward(features, mode, params)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        tf.logging.info('\n'+pprint.pformat(tf.trainable_variables())) 
        
        loss_op = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,
                                                                        labels=labels))
        
        train_op = tf.train.AdamOptimizer().minimize(loss_op,
                                                     global_step = tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode = mode,
                                          loss = loss_op,
                                          train_op = train_op)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions = tf.sigmoid(logits))

In [ ]:
triples_all, triples_tr, triples_va, triples_te = load_triple()
e2idx, p2idx, params = build_vocab(triples_all)
s2p2o = build_multi_label(triples_tr)

model = tf.estimator.Estimator(model_fn,
                               params = params)

for n_epoch in range(Config.n_epochs):
    model.train(lambda: train_input_fn(triples_tr, e2idx, p2idx, s2p2o))
    evaluate_rank(model,
                  triples_va,
                  triples_te,
                  triples_all,
                  e2idx,
                  p2idx,)

Files Already Downloaded
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpxph7wo9a', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11aaed668>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 150) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 150) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bi

100%|████████████████████████████| 5000/5000 [00:43<00:00, 113.74it/s]


[test] Raw Mean Rank: 18437.9084
[test] Raw Hits@1: 0.4
[test] Raw Hits@3: 0.58
[test] Raw Hits@5: 0.96
[test] Raw Hits@10: 1.7000000000000002
[test] Filtered Mean Rank: 18422.7802
[test] Filtered Hits@1: 0.4
[test] Filtered Hits@3: 0.58
[test] Filtered Hits@5: 0.96
[test] Filtered Hits@10: 1.7000000000000002
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 150) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 150) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(7488, 150) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(150,) dtype=float32_ref>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpxph7wo9a/model.ckpt-1179
INFO:te

100%|█████████████████████████████| 5000/5000 [00:51<00:00, 97.85it/s]


[test] Raw Mean Rank: 17029.135
[test] Raw Hits@1: 0.48
[test] Raw Hits@3: 1.28
[test] Raw Hits@5: 1.7000000000000002
[test] Raw Hits@10: 2.48
[test] Filtered Mean Rank: 17013.9658
[test] Filtered Hits@1: 0.48
[test] Filtered Hits@3: 1.28
[test] Filtered Hits@5: 1.7000000000000002
[test] Filtered Hits@10: 2.48
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 150) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 150) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(7488, 150) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(150,) dtype=float32_ref>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpxph7wo9a/model.ckpt-2358
INFO:t

100%|████████████████████████████| 5000/5000 [00:48<00:00, 103.03it/s]


[test] Raw Mean Rank: 15989.7194
[test] Raw Hits@1: 0.76
[test] Raw Hits@3: 1.8399999999999999
[test] Raw Hits@5: 2.46
[test] Raw Hits@10: 3.4000000000000004
[test] Filtered Mean Rank: 15974.411
[test] Filtered Hits@1: 0.8200000000000001
[test] Filtered Hits@3: 1.8800000000000001
[test] Filtered Hits@5: 2.52
[test] Filtered Hits@10: 3.46
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 150) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 150) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(7488, 150) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(150,) dtype=float32_ref>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpxph